### Script for downloading 3d image volume from bossdb.org and storing zarr or n5

In [8]:
from intern import array

channel = array("bossdb://kuan_phelps2020/drosophila_brain_120nm/drBrain_120nm_rec", dtype="uint8", resolution=0)

In [2]:
channel.shape

(3200, 3584, 3584)

In [3]:
channel.voxel_size, channel.axis_order

((np.float64(120.0), np.float64(120.0), np.float64(120.0)), 'ZYX')

In [4]:
import tensorstore as ts
import numpy as np

In [5]:
zarr_out = ts.open({
    "driver": "zarr3",
    "kvstore": {
        "driver": "file",
        "path": "/home/katzw/image-conversion/data.zarr",
    },
    # "metadata": {
    #     "shape": [channel.shape[0], channel.shape[1], channel.shape[2]],
    #     "chunk_grid": {
    #         "name": "regular", 
    #         "configuration": {"chunk_shape": [160, 160, 160]}
    #     },
    #     "chunk_key_encoding": {"name": "default"},
    #     # "codecs": [
    #     #     {
    #     #         "name": "blosc", 
    #     #         "configuration": {"cname": "lz4", "clevel": 5}
    #     #     }
    #     # ],
    #     "data_type": "uint8"
    # },
    # "create": True,
    # "delete_existing": True
}).result()

In [7]:
n5_out = ts.open({
    'driver': 'n5',
    'kvstore': {
        'driver': 'file',
        'path': '/home/katzw/image-conversion/data.n5',
    },
    'metadata': {
        'compression': {
            'type': 'gzip'
        },
        'dataType': 'uint8',
        'dimensions': [channel.shape[0], channel.shape[1], channel.shape[2]],
        'blockSize': [160, 160, 160],
    },
    'create': True,
    'delete_existing': True,
}).result()

In [9]:
chunk_size = (160, 160, 160)

async def write_chunks(channel, array_out, chunk_size, start_indices=(0, 0, 0)):
    z_chunks, y_chunks, x_chunks = [range(start, dim, chunk) for start, dim, chunk in zip(start_indices, channel.shape, chunk_size)]
    
    for z0 in z_chunks:
        for y0 in y_chunks:
            for x0 in x_chunks:
                z1 = min(z0 + chunk_size[0], channel.shape[0])
                y1 = min(y0 + chunk_size[1], channel.shape[1])
                x1 = min(x0 + chunk_size[2], channel.shape[2])
                
                # Extract the chunk from the channel
                chunk = channel[z0:z1, y0:y1, x0:x1]
                
                # Write the chunk to array_out
                await array_out[z0:z1, y0:y1, x0:x1].write(chunk)
                print(f"wrote chunk {z0}:{z1}, {y0}:{y1}, {x0}:{x1}")

In [ ]:
start_indices = (0, 0, 0) # allows starting from different chunk if aborted before completion
await write_chunks(channel, zarr_out, chunk_size, start_indices)